Description des entreprises qui divulguent leurs données de gender pay gap.  ? Bons/mauvais élèves dans leurs secteurs sur les autres variables ? Biais de taille (seules grosses entreprises divulguent) ? Analyse en composante principales (ACP) pour faire ressortir les principales variables discriminantes ?


In [9]:
!pip install openpyxl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 8.0 MB/s eta 0:00:00


In [10]:
import pandas as pd
import numpy as np
import math
import openpyxl


In [14]:
file_imp_path = "data/database-imputed ESG.xlsx"
file_sect = "data/secteurs.csv"
corresp_secteurs = pd.read_csv(file_sect, sep=";")
imp_data = pd.read_excel(file_imp_path)
imp_data = imp_data.join(corresp_secteurs.set_index('primary_industry'), on = "primary_industry")

FileNotFoundError: [Errno 2] No such file or directory: 'data/secteurs.csv'

In [28]:
imp_data = pd.read_excel(file_imp_path)

imp_data['missing'] = imp_data['gender_pay_gap'].isnull().astype(float)

count_missing = imp_data[imp_data['missing'] == 1]['missing'].count()
print("Number of missing values in 'gender_pay_gap':", count_missing)

missing_paygap_subdataset = imp_data[imp_data['missing'] == 1]
non_missing_paygap_subdataset = imp_data[imp_data['missing'] == 0]

Number of missing values in 'gender_pay_gap': 0


# Secteurs de prédilection


In [33]:
# Attendre le fichier de Aymeric

imp_data['gender_pay_gap'].head(200)


0      13.506805
1      13.869499
2      21.097915
3      -1.690225
4      19.982123
         ...    
195     5.121436
196     6.014321
197     3.502993
198     4.255374
199    -0.111596
Name: gender_pay_gap, Length: 200, dtype: float64

 # Biais de taille (seules grosses entreprises divulguent) ?


# Bons/mauvais élèves sur les autres variables en controlant du secteur, de la région, de la taille ?

In [20]:
# Variables disponibles
# variables_disponibles = ["market_cap", "employees", "revenue", "scope_1", "scope_2", "scope_3",
#                         "waste_production", "waste_recycling", "water_consumption",
#                         "water_withdrawal", "energy_consumption", "hours_of_training",
#                         "gender_pay_gap", "independent_board_members_percentage",
#                         "legal_costs_paid_for_controversies", "ceo_compensation"]

# Autres varia sociales : 
# Hours of training per employee 
# ceo compensation

# varia ecolo:
# scope 1/2/3




,name_year,company_id,year,company_name,ticker,lei_cici,isin,business_desc,region,hq_country,...,waste_recycling,water_consumption,water_withdrawal,energy_consumption,hours_of_training,gender_pay_gap,independent_board_members_percentage,legal_costs_paid_for_controversies,ceo_compensation,missing
0,Apple Inc._2020,c67ee84d-5cf4-4cd0-85b9-027ad082748f,2020,Apple Inc.,NasdaqGS:AAPL,HWUPKR0MPOU8FGXBT394,US0378331005,"Apple Inc. designs, manufactures, and markets ...",United States and Canada,United States,...,28939.8200,4.875000e+06,4.875000e+06,3.019170e+06,144885.5,13.506805,0.008571,0.000000,1.476926e+07,0.0
1,Apple Inc._2019,c67ee84d-5cf4-4cd0-85b9-027ad082748f,2019,Apple Inc.,NasdaqGS:AAPL,HWUPKR0MPOU8FGXBT394,US0378331005,"Apple Inc. designs, manufactures, and markets ...",United States and Canada,United States,...,32806.4100,4.890152e+06,4.992424e+06,2.889680e+06,144885.5,13.869499,0.008571,1.850000,1.155547e+07,0.0
2,Apple Inc._2018,c67ee84d-5cf4-4cd0-85b9-027ad082748f,2018,Apple Inc.,NasdaqGS:AAPL,HWUPKR0MPOU8FGXBT394,US0378331005,"Apple Inc. designs, manufactures, and markets ...",United States and Canada,United States,...,30104.5941,4.765152e+06,4.924242e+06,2.676460e+06,144885.5,21.097915,0.008750,920.013001,1.568222e+07,0.0
3,Saudi Arabian Oil Company_2020,9ea5e71e-33ac-4916-9c58-50ce9fa54abe,2020,Saudi Arabian Oil Company,SASE:2222,5586006WD91QHB7J4X50,SA14TG012N13,Saudi Arabian Oil Company operates as an integ...,Africa / Middle East,Saudi Arabia,...,15309.0000,3.615000e+07,3.615000e+07,1.495640e+08,144885.5,-1.690225,0.454540,0.000000,2.199049e+08,0.0
4,Saudi Arabian Oil Company_2019,9ea5e71e-33ac-4916-9c58-50ce9fa54abe,2019,Saudi Arabian Oil Company,SASE:2222,5586006WD91QHB7J4X50,SA14TG012N13,Saudi Arabian Oil Company operates as an integ...,Africa / Middle East,Saudi Arabia,...,15309.0000,7.842800e+08,3.104395e+09,1.212160e+08,144885.5,19.982123,0.454540,1228.533586,2.015694e+07,0.0
